# 0. Introduction

This notebook contains the exercise presented for the Introduction to Data Science section of Udacity's Data Science nanodegree. The course project is called Writing a Data Scientist Blog Post and consists in answering 3 questions based on a dataset selected by the student. The findings need to be presented as a blog post.

I have picked the following Kaggle competition dataset: House Prices - Advanced Regression Techniques. Predict sales prices and practice feature engineering, RFs, and gradient boosting. The project is described on kaggle's website as follows:

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

# 1. Dataset exploration

The competition includes 2 different datasets named train.csv and test.csv. The test dataset doesn't include the price column. The predictions for the test dataset need to be submitted to kaggle in order to obtain a competition score. For this project, only the train dataset will be used and it will be subdivided into train/test subsets for the modeling part of the exercise.

The following cells include a first assessment of the dataset.

In [1]:
#import all the necessary packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#load the train dataset

df = pd.read_csv('./train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df.shape

(1460, 81)

As specified in the competition description, the dataset includes 79 independent variables and a dependent variable (price). That makes a total of 81 columns including the Id. There are a total of 1460 entries.

In [4]:
df['Id'].is_unique

True

We see there are no duplicates in the Id column, so we can assume all rows are independent from each other and correspond to individual entries.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

We see in the previous cell that all the columns seem to have a correct data type associated. All numerical variables are either integers or floats and the rest are categorical. We also see there are NaN values in many of the columns, which I look at in more detail in the following cells.

In [6]:
no_nulls = df.columns[df.isnull().mean()==0]
no_nulls.size

62

In [7]:
half_nulls = df.columns[df.isnull().mean() > 0.5]
half_nulls.size

4

Out of the 81 columns, 62 of them have no NaN values. 4 columns have more than 50% missing values and the rest have less than 50% NaN. From the list above I can look separately into each of them to decide what to do with the missing values.

In [8]:
df.Alley.value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

For instance, the _Alley_ column has only a value assigned to the rows corresponding to houses with alleys. Houses without alley access present a NaN value. In this case I'll create a new column named _Alley_acces_ with Yes No values as this could be an important feature to consider. I will also fill the NaN of the original columns with 'No Alley' values. I will do the same for the _Fence_ column as well as for the columns referring to the garage or basement.

In [9]:
df.loc[df['Alley'] == 'Grvl', 'Has_Alley'] = 'Yes'
df.loc[df['Alley'] == 'Pave', 'Has_Alley'] = 'Yes'
df.Has_Alley.fillna('No', inplace=True)
df.Has_Alley.value_counts()

No     1369
Yes      91
Name: Has_Alley, dtype: int64

In [10]:
df.Alley.fillna('No_Alley', inplace=True)
df.Alley.value_counts()

No_Alley    1369
Grvl          50
Pave          41
Name: Alley, dtype: int64

In [11]:
df.Fence.value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [12]:
df.Fence.fillna('No_Fence', inplace=True)
df.loc[df['Fence'] == 'No_Fence', 'Has_Fence'] = 'No'
df.Has_Fence.fillna('Yes', inplace=True)
df.Has_Fence.value_counts()

No     1179
Yes     281
Name: Has_Fence, dtype: int64

In [13]:
df.BsmtCond.fillna('No_Basement', inplace=True)
df.loc[df['BsmtCond'] == 'No_Basement', 'Has_Bsmt'] = 'No'

df.BsmtQual.fillna('No_Basement', inplace=True)

df.BsmtFinType1.fillna('No_Basement', inplace=True)

df.BsmtExposure.fillna('No_Basement', inplace=True)

df.BsmtFinType2.fillna('No_Basement', inplace=True)

df.Has_Bsmt.fillna('Yes', inplace=True)
df.Has_Bsmt.value_counts()

Yes    1423
No       37
Name: Has_Bsmt, dtype: int64

In [14]:
df.GarageType.fillna('No_Garage', inplace=True)
df.loc[df['GarageType'] == 'No_Garage', 'Has_Garage'] = 'No'

df.GarageYrBlt.fillna('No_Garage', inplace=True)
df.GarageFinish.fillna('No_Garage', inplace=True)
df.GarageQual.fillna('No_Garage', inplace=True)
df.GarageCond.fillna('No_Garage', inplace=True)

df.Has_Garage.fillna('Yes', inplace=True)
df.Has_Garage.value_counts()

Yes    1379
No       81
Name: Has_Garage, dtype: int64

Given the low number of houses with a pool in the dataset, I remove the _Pool quality_ column. The fact of containing a pool can be important, but it's already included in the PoolArea column which is kept.

I also remove the _Miscellaneous feature_ column as only 54 houses have a value in it and the listed features are very different from one another. Similarly, I'll delete the fireplace quality column as there are almost half the values missing.

In [17]:
df = df.drop(columns=['PoolQC', 'MiscFeature', 'FireplaceQu'])

I will fill the NaN values of the 8 rows in the Masonry Veneer columns, with 'None' and 0, which are values that are already included in the existing column values:

In [ ]:
df.MasVnrArea.fillna(0, inplace=True)
df.MasVnrType.fillna('None', inplace=True)

For the _Electrical_ column I use the mode to fill the only row with a missing value:

In [19]:
df.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [20]:
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace=True)
df.Electrical.value_counts()

SBrkr    1335
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

Finally, for _LotFrontage_ I'll fill the missing values with the average of the column, as all the lots should have some street frontage.

In [24]:
df['LotFrontage'].fillna(df['LotFrontage'].mean(), inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

The modified dataset contains no missing values and is ready for modeling in the next steps.

# 2. Business Questions

After a first assessment of the dataset I would like to answer the following questions:

A. Does the dataset reflect the financial crisis of 2007-2008. Did it have a significant impact on the price of the houses sold during and after that period?

B. How well can we predict prices based on all the features contained in the dataset?

C. What are the top 5 features in terms of impact weight on the house price? How well can we predict prices based only on these 5 features?